Implementation of SVM with a Gaussian Kernel

In [7]:
import numpy as np

The formula used for the kernel is : $$K(x, y) = \exp\left(-\gamma \|\mathbf{x} - \mathbf{y}\|^2\right), \quad \text{where } \|\mathbf{x} - \mathbf{y}\|^2 = \sum_{i=1}^{n} (x_i - y_i)^2$$

In [8]:
class SVMRBF:
    def __init__(self, C=1.0, tol=1e-3, max_iter=1000, gamma=0.5):
        self.C = C
        self.tol = tol
        self.max_iter = max_iter
        self.gamma = gamma
        self.alpha = None
        self.b = 0
        self.X = None
        self.y = None

    def kernel(self, x1, x2):
        # Gaussian Kernel (RBF): K(x1, x2) = exp(-gamma * ||x1 - x2||^2)
        diff = x1 - x2
        return np.exp(-self.gamma * np.dot(diff, diff))

    def fit(self, X, y):
        n_samples, n_features = X.shape
        self.X = X
        self.y = y
        self.alpha = np.zeros(n_samples)
        self.b = 0

        passes = 0
        while passes < self.max_iter:
            num_changed_alphas = 0
            for i in range(n_samples):
                # Compute f(x_i)
                f_i = np.sum(self.alpha * y * np.array([self.kernel(X[j], X[i]) for j in range(n_samples)])) + self.b
                E_i = f_i - y[i]

                if ((y[i]*E_i < -self.tol and self.alpha[i] < self.C) or
                    (y[i]*E_i > self.tol and self.alpha[i] > 0)):

                    j = np.random.choice([x for x in range(n_samples) if x != i])
                    f_j = np.sum(self.alpha * y * np.array([self.kernel(X[k], X[j]) for k in range(n_samples)])) + self.b
                    E_j = f_j - y[j]

                    alpha_i_old, alpha_j_old = self.alpha[i], self.alpha[j]

                    if y[i] != y[j]:
                        L = max(0, self.alpha[j] - self.alpha[i])
                        H = min(self.C, self.C + self.alpha[j] - self.alpha[i])
                    else:
                        L = max(0, self.alpha[i] + self.alpha[j] - self.C)
                        H = min(self.C, self.alpha[i] + self.alpha[j])
                    if L == H:
                        continue

                    K_ii = self.kernel(X[i], X[i])
                    K_jj = self.kernel(X[j], X[j])
                    K_ij = self.kernel(X[i], X[j])
                    eta = 2 * K_ij - K_ii - K_jj
                    if eta >= 0:
                        continue

                    self.alpha[j] = self.alpha[j] - (y[j]*(E_i - E_j)) / eta
                    self.alpha[j] = np.clip(self.alpha[j], L, H)

                    if abs(self.alpha[j] - alpha_j_old) < 1e-5:
                        continue

                    self.alpha[i] = self.alpha[i] + y[i]*y[j]*(alpha_j_old - self.alpha[j])

                    b1 = self.b - E_i - y[i]*(self.alpha[i]-alpha_i_old)*K_ii - y[j]*(self.alpha[j]-alpha_j_old)*K_ij
                    b2 = self.b - E_j - y[i]*(self.alpha[i]-alpha_i_old)*K_ij - y[j]*(self.alpha[j]-alpha_j_old)*K_jj

                    if 0 < self.alpha[i] < self.C:
                        self.b = b1
                    elif 0 < self.alpha[j] < self.C:
                        self.b = b2
                    else:
                        self.b = (b1 + b2) / 2

                    num_changed_alphas += 1

            if num_changed_alphas == 0:
                passes += 1
            else:
                passes = 0

        self.support_vectors_idx = np.where(self.alpha > 1e-5)[0]
        self.support_vectors = self.X[self.support_vectors_idx]
        self.support_vector_labels = self.y[self.support_vectors_idx]
        self.alpha = self.alpha[self.support_vectors_idx]

    def project(self, X):
        y_predict = np.zeros(X.shape[0])
        for i in range(X.shape[0]):
            s = 0
            for alpha, sv_y, sv in zip(self.alpha, self.support_vector_labels, self.support_vectors):
                s += alpha * sv_y * self.kernel(sv, X[i])
            y_predict[i] = s
        return y_predict + self.b

    def predict(self, X):
        return np.sign(self.project(X))

def compute_metrics(y_true, y_pred):
  accuracy = np.mean(y_true == y_pred)
  tp = np.sum((y_true == 1) & (y_pred == 1))
  tn = np.sum((y_true == -1) & (y_pred == -1))
  fp = np.sum((y_true == -1) & (y_pred == 1))
  fn = np.sum((y_true == 1) & (y_pred == -1))

  precision = tp / (tp + fp) if (tp+fp) > 0 else 0
  recall = tp / (tp + fn) if (tp+fn) > 0 else 0
  f1_score = 2 * precision * recall / (precision + recall) if (precision+recall) > 0 else 0

  return accuracy, precision, recall, f1_score


In [9]:
if __name__ == "__main__":
    # Ejemplo simple: datos no linealmente separables
    np.random.seed(42)
    n_samples = 50
    theta = np.linspace(0, 2*np.pi, n_samples)
    X1 = np.array([np.cos(theta), np.sin(theta)]).T + np.random.randn(n_samples, 2)*0.1
    X2 = np.array([2*np.cos(theta), 2*np.sin(theta)]).T + np.random.randn(n_samples, 2)*0.1
    X = np.vstack((X1, X2))
    y = np.hstack((np.ones(n_samples), -np.ones(n_samples)))

    model = SVMRBF(C=1.0, gamma=1.0)
    model.fit(X, y)
    preds = model.predict(X)

    accuracy, precision, recall, f1_score = compute_metrics(y, preds)
    print("Predicciones:", preds)
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1_score:.4f}")

Predicciones: [ 1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.
  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1.  1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.
 -1. -1. -1. -1. -1. -1. -1. -1. -1. -1.]
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1 Score: 1.0000
